In [9]:
import pandas as pd

In [10]:
df = pd.read_excel("skincare catalog.xlsx")
df.head()

,product_id,name,category,description,top_ingredients,tags,price (USD),margin (%)
0,SRM001,Radiant Renewal Serum,Serum,Brightens dull skin and smooths fine lines for...,Ascorbic Acid (Vitamin C); Hyaluronic Acid; Ni...,antiaging|brightening|hydration,72,0.48
1,SRM002,Dewdrop Hydration Elixir,Serum,Water-weight serum that drenches skin in long-...,Hyaluronic Acid; Panthenol; Vitamin F (Linolei...,hydration|barrier-repair|dry-skin,58,0.42
2,SRM003,Clear Slate BHA Serum,Serum,Clarifies congested pores and keeps oil in che...,Salicylic Acid; Zinc PCA; Green Tea Extract,acne-prone|oily-control|pore-care,46,0.40
3,SRM004,Midnight Peptide Firming Serum,Serum,Overnight peptide complex visibly firms and no...,Palmitoyl Tripeptide-5; Soluble Collagen; Vita...,antiaging|firming|dry-skin,85,0.52
4,SRM005,Calm & Soothe Sensitive Serum,Serum,Lightweight gel serum that calms redness and s...,Centella Asiatica; Allantoin; Beta-Glucan,sensitive|redness-relief|hydration,49,0.38


In [11]:
df['category'].value_counts()

category
Serum                  10
Cream / Moisturizer    10
Toner                   5
Face Mask               3
Body Wash               3
Conditioner             3
Shampoo                 3
Sunscreen               1
Hair Mask               1
Name: count, dtype: int64

In [17]:
df['top_ingredients'].str.split('; ').explode().value_counts().reset_index().sort_values(['top_ingredients']).to_csv('ingredients.csv', header=True, index=True)

In [4]:
df['top_ingredients'].str.split('; ').explode().unique()

array(['Ascorbic Acid (Vitamin C)', 'Hyaluronic Acid', 'Niacinamide',
       'Panthenol', 'Vitamin F (Linoleic Acid)', 'Salicylic Acid',
       'Zinc PCA', 'Green Tea Extract', 'Palmitoyl Tripeptide-5',
       'Soluble Collagen', 'Vitamin E', 'Centella Asiatica', 'Allantoin',
       'Beta-Glucan', 'Tranexamic Acid', 'Licorice-Root Extract',
       'Tremella Mushroom Extract', 'Willow-Bark', 'Ceramides NP+AP+EOP',
       'Vitamin F', 'Squalane', 'Retinaldehyde', 'Bakuchiol',
       'Ferulic Acid', 'Damask Rose Water', 'Glycerin', 'Aloe Vera Juice',
       'Salicylic Acid 2%', 'Witch Hazel', 'Glycolic Acid 5%',
       'Lactic Acid', 'Rooibos Extract', 'Cucumber Water',
       'Gluconolactone (PHA)', 'Zinc Oxide (SPF 30)', 'Retinol',
       'Peptides', 'Vitamin C', 'Caffeine', 'Shea Butter', 'Cocoa Butter',
       'Palmitoyl Tripeptide-1', 'Cholesterol', 'Titanium Dioxide',
       'Zinc Oxide (SPF 50)', 'Aloe Vera', 'Arctic Oat Lipids',
       'Alpha-Arbutin', 'Licorice Root', 'Helichrysu

In [14]:
df.set_index('product_id')['name'].to_dict()

{'SRM001': 'Radiant Renewal Serum',
 'SRM002': 'Dewdrop Hydration Elixir',
 'SRM003': 'Clear Slate BHA Serum',
 'SRM004': 'Midnight Peptide Firming Serum',
 'SRM005': 'Calm & Soothe Sensitive Serum',
 'SRM006': 'Flash Fade Spot Corrector',
 'SRM007': 'Velvet Matte Pore Serum',
 'SRM008': 'AquaShield Barrier Booster',
 'SRM009': 'Sunrise Retinal Serum',
 'SRM010': 'Glow Guard Antioxidant Serum',
 'TNR001': 'Rosewater Rescue Hydrating Toner',
 'TNR002': 'ClearWave 2% BHA Toner',
 'TNR003': 'GlowPrep 5% Glycolic Toner',
 'TNR004': 'CalmCucumber pH Mist',
 'TNR005': 'SmoothStart PHA Tonic',
 'CRM001': 'HydraCloud Daily Gel-Cream SPF 30',
 'CRM002': 'Velvet Night Repair Cream',
 'CRM003': 'Radiance Spot-Bright Eye Cream',
 'CRM004': 'Deep Nourish Body Butter',
 'CRM005': 'SmoothLift Wrinkle Corrector',
 'CRM006': 'BarrierFix Ceramide Cream',
 'CRM007': 'ClearMatte Oil-Control Gel Moisturizer',
 'CRM008': 'SunShield Mineral Moisturizer SPF 50',
 'CRM009': 'Arctic Hand Rescue Cream',
 'CRM010

In [21]:
from rapidfuzz import process, fuzz
import re
# --- Utility: Fuzzy match product in text ---
def extract_product_from_text(text, catalog_products: dict, threshold=80):
    """
    Uses fuzzy matching to find a product name from the catalog within a given text
    and returns the corresponding product ID.

    Args:
        text (str): The input text to search within.
        catalog_products (dict): A dictionary where keys are product IDs and values are product names.
                                 Example: {'prod_1': 'HydraCloud Daily Gel-Cream SPF 30'}
        threshold (int): The minimum fuzzy matching score (0-100) to consider a match.

    Returns:
        str: The ID of the matched product, or None if no match is found above the threshold.
    """
    if not text or not catalog_products:
        return None
    # Create a list of product names (choices) for fuzzy matching
    # and a mapping from lowercased name back to original ID
    product_names_lower_to_id = {}
    choices_for_fuzzy_match = []

    for product_id, product_name in catalog_products.items():
        # cleaned_product_name = re.sub(r'[^\w\s]', '', product_name).lower()
        product_names_lower_to_id[product_name.lower()] = product_id
        choices_for_fuzzy_match.append(product_name.lower())
    try:
        match, score, _ = process.extractOne(text.lower(), choices_for_fuzzy_match, scorer=fuzz.partial_token_set_ratio)
        if score >= threshold:
            return product_names_lower_to_id.get(match)
        return None
    except Exception as e:
         return None


# --- Utility: Normalize star ratings ---
def normalize_rating(rating):
    """
    Normalizes star ratings from various formats to 'X out of 5'.
    """
    rating_str = str(rating).strip()
    if not rating_str:
        return "N/A"
    # Handle star strings like "★★★☆☆"
    if set(rating_str) <= set("★☆") and len(rating_str) == 5:
        stars = rating_str.count("★")
        return f"{stars} out of 5"
    # Handle numeric or "4/5" style
    try:
        rating_num = int(float(rating_str.split("/")[0]))
        if 0 <= rating_num <= 5:
             return f"{rating_num} out of 5"
    except Exception:
        pass # Fallback to returning original string if parsing fails

    return rating_str


In [4]:
# --- Step 1: Load Catalog for Product Linking ---
def load_catalog_products(path):
    """
    Loads the skincare catalog Excel file and returns a set of product names (lowercase) for linking.
    """
    try:
        df = pd.read_excel(path)
        return df.set_index('product_id')['name'].to_dict()
    except FileNotFoundError:
        raise
    except KeyError:
        raise
    except Exception as e:
        raise
catalog_products = load_catalog_products('skincare catalog.xlsx')

In [1]:
feedback_xlsx = 'CustomerFeedback.xlsx'

In [25]:
df_reviews = pd.read_excel(feedback_xlsx, sheet_name="Reviews").dropna()
for index, row in df_reviews.iterrows():
    try:
        reviewer = str(row.get("Reviewer", "")).strip()
        product_name = str(row.get("Product", "")).strip()
        review = str(row.get("Review", "")).strip()
        rating = normalize_rating(row.get("Rating", ""))
        # Link to catalog
        product_id = extract_product_from_text(product_name, catalog_products)
        if not product_id: print(row.to_dict())
    except Exception as e:
        continue # Skip this row and continue with the next


In [27]:
process.extractOne('“Package arrived leaking—Velvet Night Repair cap was loose and product everywhere.”'.lower(), map(str.lower, catalog_products.values()), scorer=fuzz.partial_token_set_ratio )

('velvet night repair cream', 100.0, 16)

In [24]:
extract_product_from_text('“Package arrived leaking—Velvet Night Repair cap was loose and product everywhere.”', catalog_products)

In [23]:

df_tickets = pd.read_excel(feedback_xlsx, sheet_name="Customer Support Tickets").dropna()
for index, row in df_tickets.iterrows():
    try:
        ticket_id = str(row.get("Ticket ID", "")).strip()
        customer_msg = str(row.get("Customer Message", "")).strip()
        support_resp = str(row.get("Support Response", "")).strip()
        # Try to extract product from customer message or support response
        product_id = extract_product_from_text(customer_msg, catalog_products) or extract_product_from_text(support_resp, catalog_products) or ""
        in_catalog = bool(product_id) # True if a product was extracted
        
        if not in_catalog: print(row.to_dict())
    except Exception as e:
        continue # Skip this row and continue with the next


{'Ticket ID': 'CS005', 'Customer Message': '“Package arrived leaking—Velvet Night Repair cap was loose and product everywhere.”', 'Support Response': '“So sorry! We’ve processed a no-cost replacement shipping out today (#RPL-6721) and flagged the lot for QA. No need to return the damaged jar.”'}
{'Ticket ID': 'CS009', 'Customer Message': '“Tracking shows my order stuck in ‘label created’ for five days. Vacation starts soon—help!”', 'Support Response': '“Apologies for the delay; high carrier volume caused a backlog. We’ve upgraded you to 2-Day Air at no cost (new tracking # 1Z339…); ETA now Apr 8.”'}


In [12]:
import os
import shutil
import random

import glob
from pathlib import Path

In [9]:
choices = ['SRM001', 'SRM002', 'SRM003']
# os.listdir("../frontend/public/products")

In [13]:
for p in df[~df['product_id'].isin(choices)]['product_id']:
    choice = random.choice(choices)
    shutil.copytree('../frontend/public/products/'+choice, '../frontend/public/products/'+p)

In [18]:
products_dir = Path('../frontend/public/products')
product_dirs = [d for d in products_dir.iterdir() if d.is_dir()]
product_dirs.sort()
product_dirs[0]

WindowsPath('../frontend/public/products/BW001')

In [20]:
total_renamed = 0
for product_dir in product_dirs:
    product_id = product_dir.name
    print(f"Processing product: {product_id}")
    
    # Get all PNG files in the product directory
    png_files = list(product_dir.glob('*.png'))
    
    if not png_files:
        print(f"  No PNG files found in {product_id}")
        continue
    
    # Rename each image with index
    for index, png_file in enumerate(png_files, 1):
        old_name = png_file.name
        new_name = f"{index}.png"
        new_path = product_dir / new_name
        
        try:
            # Check if target file already exists
            if new_path.exists():
                print(f"  Warning: {new_name} already exists, skipping {old_name}")
                continue
            
            # Rename the file
            png_file.rename(new_path)
            print(f"  Renamed: {old_name} -> {new_name}")
            total_renamed += 1
            
        except Exception as e:
            print(f"  Error renaming {old_name}: {e}")

Processing product: BW001
Processing product: BW002
Processing product: BW003
Processing product: CRM001
Processing product: CRM002
Processing product: CRM003
Processing product: CRM004
Processing product: CRM005
Processing product: CRM006
Processing product: CRM007
Processing product: CRM008
Processing product: CRM009
Processing product: CRM010
Processing product: HC001
Processing product: HC002
Processing product: HC003
Processing product: HC004
Processing product: HC005
Processing product: HC006
Processing product: HC007
Processing product: MSK001
Processing product: MSK002
Processing product: MSK003
Processing product: SNS001
Processing product: SRM001
Processing product: SRM002
Processing product: SRM003
Processing product: SRM004
Processing product: SRM005
Processing product: SRM006
Processing product: SRM007
Processing product: SRM008
Processing product: SRM009
Processing product: SRM010
Processing product: TNR001
Processing product: TNR002
Processing product: TNR003
Processing 